In [1]:
%reset -f
import pandas as pd
import numpy as np
from wavhandler import *
from utils_train import *

data1 = Dataset('Wingbeats')
data1.read(data='all', setting='read', labels='text', loadmat=False)

make_classification_conv2d(data1.filenames.tolist(), data1.y.tolist(), model_name='mosquitos', setting='stft',
                          undersampling=False)

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.
Using TensorFlow backend.


Data: all.
Read 279566 filenames in 0.81 seconds.


Class balance: 
0    85553
5    74599
3    49471
4    30415
1    20231
2    19297
Name: 0, dtype: int64

Epoch 1/100
  34/6291 [..............................] - ETA: 1:14:39 - loss: 1.3224 - acc: 0.5119

KeyboardInterrupt: 